In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
# import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [10]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [11]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [12]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [13]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test_original = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [14]:
X.shape

(2013, 52)

In [15]:
X.columns

Index(['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'auto parts', 'books and music', 'clothing',
       'computers', 'furniture', 'home goods', 'jewelry', 'mobile',
       'office equipment', 'sporting goods', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_excee

In [16]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']

train_features = X[col_names]
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.transform(train_features.values)
X[col_names] = train_features

test_features = X_test_original[col_names]
scaler_test = StandardScaler().fit(test_features.values)
test_features = scaler_test.transform(test_features.values)
X_test_original[col_names] = test_features

In [17]:
selected_features = ['num_bids_per_ip', 'num_bids_per_auction', 'num_bids_per_country', 'mean_ip_per_auction', 'perc_inst_resp', 'time', 'mean_url_per_auction', 'inst_resp', 'num_bids', 'time_to_bid']
X = X[selected_features]
X_test_original = X_test_original[selected_features]

### Helper Functions

In [19]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [20]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### SKFold + SMOTE (Base Model)

In [21]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
counter = 1
dt_result = []
rf_result = []

over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    # decision tree
    print("DECISION TREE")
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    dt_result.append(evaluate(dt, X_train, y_train, X_test, y_test))
    
    # random forest
    print("RANDOM FOREST")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    rf_result.append(evaluate(rf,X_train,y_train,X_test,y_test))

Fold number 1
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9666    0.9084    0.9366       382
         1.0     0.2045    0.4286    0.2769        21

    accuracy                         0.8834       403
   macro avg     0.5856    0.6685    0.6067       403
weighted avg     0.9269    0.8834    0.9022       403

FBeta Score
0.35156249999999994
Model Performance
Accuracy = 0.8834%.
AUC ROC = 0.6685%.


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9862    0.9346    0.9597       382
         1.0     0.3750    0.7500    0.5000        20

    accuracy                         0.9254       402
   macro avg     0.6806    0.8423    0.7298       402
weighted avg     0.9558    0.9254    0.9368       402

FBeta Score
0.625
Model Performance
Accuracy = 0.9254%.
AUC ROC = 0.9084%.
******************************************

In [22]:
process_results(dt_result)

final train accuracy: 1.0
final train AUC: 1.0
final train fbeta: 1.0
final test accuracy: 0.8917052454847351
final test AUC: 0.7321166791323859
final test fbeta: 0.44300541289810375


In [23]:
process_results(rf_result)

final train accuracy: 0.9998125585754452
final train AUC: 1.0
final train fbeta: 0.999475065616798
final test accuracy: 0.9130587756009035
final test AUC: 0.9004325604587384
final test fbeta: 0.5230272089400804


### SKFold + SMOTE (Random Search)

In [24]:
def random_search_dt(X, y):
    print("RANDOM SEARCH DECISION TREE EXPERIMENT")
    criterion = ['gini', 'entropy']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [1,2,5,8]
    min_samples_leaf = [1,11,21,31]
    max_features = [5,10,15,25]
    min_impurity_decrease = [0.00005,0.0005,0.005,0.05]

    dt_random_grid = {'criterion': criterion,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'max_features': max_features,
                   'min_samples_leaf': min_samples_leaf, 
                   'min_impurity_decrease': min_impurity_decrease}
    
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_random = RandomizedSearchCV(estimator = dt, param_distributions = dt_random_grid, n_iter = 100, cv = skf, 
                               verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    dt_random.fit(X_train, y_train)
    
    dt_best_random = dt_random.best_estimator_
    dt_random_accuracy = evaluate(dt_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for DT")
    print(dt_random.best_params_)
    
def random_search_rf(X, y):
    print("RANDOM SEARCH RANDOM FOREST EXPERIMENT")
    # parameters for Random Forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt',5,10,25]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4,8,10]
    bootstrap = [True, False]

    # Create the random grid
    rf_random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_random_grid, n_iter = 100, cv = skf, verbose=2, 
                               random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)
    
    rf_best_random = rf_random.best_estimator_
    rf_random_accuracy = evaluate(rf_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for RF")
    print(rf_random.best_params_)
    

In [25]:
random_search_dt(X, y)

RANDOM SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 346 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.3s finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9273    0.9398    0.9335       665
         1.0     0.8444    0.8158    0.8298       266

    accuracy                         0.9044       931
   macro avg     0.8858    0.8778    0.8817       931
weighted avg     0.9036    0.9044    0.9039       931

FBeta Score
0.8213474640423922
Model Performance
Accuracy = 0.9044%.
AUC ROC = 0.9598%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9792    0.8977    0.9367       577
         1.0     0.2133    0.5926    0.3137        27

    accuracy                         0.8841       604
   macro avg     0.5963    0.7452    0.6252       604
weighted avg     0.9450    0.8841    0.9089       604

FBeta Score
0.4371584699453551
Model Performance
Accuracy = 0.8841%.
AUC ROC = 0.8473%.
**************

In [26]:
random_search_rf(X, y)

RANDOM SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.1min finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9955    0.9985    0.9970       665
         1.0     0.9962    0.9887    0.9925       266

    accuracy                         0.9957       931
   macro avg     0.9959    0.9936    0.9947       931
weighted avg     0.9957    0.9957    0.9957       931

FBeta Score
0.990210843373494
Model Performance
Accuracy = 0.9957%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9730    0.9359    0.9541       577
         1.0     0.2449    0.4444    0.3158        27

    accuracy                         0.9139       604
   macro avg     0.6089    0.6902    0.6349       604
weighted avg     0.9404    0.9139    0.9255       604

FBeta Score
0.38216560509554137
Model Performance
Accuracy = 0.9139%.
AUC ROC = 0.8602%.
**************


*****************************************************************************************
## Best Random Search Param for DT
{'min_samples_split': 5, 'min_samples_leaf': 21, 'min_impurity_decrease': 5e-05, 'max_features': 5, 'max_depth': 110, 'criterion': 'gini'}

## Best Random Search Param for RF
{'n_estimators': 1600, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 100, 'bootstrap': False}

### SKFold + SMOTE (Grid Search)

In [27]:
def grid_search_dt(X, y):
    print("GRID SEARCH DECISION TREE EXPERIMENT")
    dt_grid = {'criterion': ['entropy'],
        'max_depth': [100, 110, 120],
        'max_features': [4, 5, 6],
        'min_impurity_decrease': [5e-06, 5e-05, 5e-04],
        'min_samples_leaf': [19,21,23],
        'min_samples_split': [4,5,6]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_grid = GridSearchCV(estimator = dt, param_grid = dt_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    dt_grid.fit(X_train, y_train)
    
    dt_best_grid = dt_grid.best_estimator_
    dt_grid_accuracy = evaluate(dt_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for DT")
    print(dt_grid.best_params_)
    
    return dt_best_grid
    
def grid_search_rf(X, y):
    print("GRID SEARCH RANDOM FOREST EXPERIMENT")
    rf_grid = {
        'bootstrap': [False],
        'max_depth': [90,100,110],
        'max_features': ['auto'],
        'min_samples_leaf': [1,2,3],
        'min_samples_split': [1,2,3],
        'n_estimators': [1500,1600,1700]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_grid = GridSearchCV(estimator = rf, param_grid = rf_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    rf_grid.fit(X_train, y_train)
    
    rf_best_grid = rf_grid.best_estimator_
    rf_grid_accuracy = evaluate(rf_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for RF")
    print(rf_grid.best_params_)
    
    return rf_best_grid

In [28]:
dt_best_grid = grid_search_dt(X, y)

GRID SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 243 candidates, totalling 1215 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1002 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 1215 out of 1215 | elapsed:    1.3s finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9138    0.9564    0.9346       665
         1.0     0.8766    0.7744    0.8224       266

    accuracy                         0.9044       931
   macro avg     0.8952    0.8654    0.8785       931
weighted avg     0.9032    0.9044    0.9025       931

FBeta Score
0.7929176289453426
Model Performance
Accuracy = 0.9044%.
AUC ROC = 0.9611%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9799    0.9272    0.9528       577
         1.0     0.2759    0.5926    0.3765        27

    accuracy                         0.9123       604
   macro avg     0.6279    0.7599    0.6646       604
weighted avg     0.9484    0.9123    0.9270       604

FBeta Score
0.48192771084337355
Model Performance
Accuracy = 0.9123%.
AUC ROC = 0.8746%.
*************

In [29]:
rf_best_grid = grid_search_rf(X, y)

GRID SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  2.6min finished


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9955    0.9985    0.9970       665
         1.0     0.9962    0.9887    0.9925       266

    accuracy                         0.9957       931
   macro avg     0.9959    0.9936    0.9947       931
weighted avg     0.9957    0.9957    0.9957       931

FBeta Score
0.990210843373494
Model Performance
Accuracy = 0.9957%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9730    0.9376    0.9550       577
         1.0     0.2500    0.4444    0.3200        27

    accuracy                         0.9156       604
   macro avg     0.6115    0.6910    0.6375       604
weighted avg     0.9407    0.9156    0.9266       604

FBeta Score
0.38461538461538464
Model Performance
Accuracy = 0.9156%.
AUC ROC = 0.8602%.
**************